In [1]:
import ROOT
from fitter import FitHandler

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x91ade90


In [2]:
default_lumi = 2000
default_hel = {
    "lumi_frac": 1.,
    "e_pol": 0.,
    "p_pol": 0.,
}

# https://arxiv.org/pdf/1506.07830
# However, https://arxiv.org/pdf/2503.19983 cites the above but assigns 0.45 to both mixed pols
# LCVision scenario uses 3 ab_inv instead of 2
ilc_250_h20_lumi = 2000
ilc_250_h20_hel = [
    {
        "lumi_frac": 0.675,
        "e_pol": -0.8,
        "p_pol": 0.3,
    },
    {
        "lumi_frac": 0.225,
        "e_pol": 0.8,
        "p_pol": -0.3,
    },
    {
        "lumi_frac": 0.05,
        "e_pol": -0.8,
        "p_pol": -0.3,
    },
    {
        "lumi_frac": 0.05,
        "e_pol": 0.8,
        "p_pol": 0.3,
    },
]
ilc_250_h20_new_hel = [
    {
        "lumi_frac": 0.45,
        "e_pol": -0.8,
        "p_pol": 0.3,
    },
    {
        "lumi_frac": 0.45,
        "e_pol": 0.8,
        "p_pol": -0.3,
    },
    {
        "lumi_frac": 0.05,
        "e_pol": -0.8,
        "p_pol": -0.3,
    },
    {
        "lumi_frac": 0.05,
        "e_pol": 0.8,
        "p_pol": 0.3,
    },
]


# run = ilc_250_h20[0]

In [3]:
conf = {
    "parameters": {
        "g1z": 0.0,
        "ka": 0.0,
        "la": 0.0,
    },
    "obs_names": [
        "O_g1z_pos_1em05",
        "O_ka_pos_1em05",
        "O_la_pos_1em05",
    ],
    "signal_cat": "4f_sw_sl_signal",
    "signal_processes": [
        "4f_sw_sl_eLpL_signal",
        "4f_sw_sl_eLpR_signal",
        "4f_sw_sl_eRpL_signal",
        "4f_sw_sl_eRpR_signal",
    ],
    "backgrounds": [
        "4f_sl_bkg",
        "4f_not_sl",
        "2f",
        "3f",
        "5f",
        "6f",
        "higgs",
    ],
    "n_bins": 65,
}
input_path = "data/histograms/full/raw_histograms.root"

In [4]:
fit_handler = FitHandler(input_path, conf)

In [5]:
# ws = fit_handler.build_model([run])
run_conf = (default_lumi, [default_hel])
# run_conf = (ilc_250_h20_lumi, ilc_250_h20_new_hel)
ws = fit_handler.build_model(*run_conf)
# coupling_pars = fit_handler.coupling_pars
coupling_pars = [ws.var(name) for name in conf["parameters"]]
obs_pars = [ws.var(f"{name}_0") for name in conf["obs_names"]]
# obs_pars = [ws.var(f"{name}_{idx}") for idx in range(len(run_conf[1])) for name in conf["obs_names"] ]

{'lumi_frac': 1.0, 'e_pol': 0.0, 'p_pol': 0.0}
hello
start constructor
finished constructor
start constructor
finished constructor
start constructor
finished constructor
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooMultiVarGaussian::multi_gauss_0
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooRealVar::O_g1z_pos_1em05_0
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooRealVar::O_ka_pos_1em05_0
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooRealVar::O_la_pos_1em05_0
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooFunctorBinding::expectation_O_g1z_pos_1em05_0
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooProduct::lumi_0
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooConstVar::1
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooRealVar::total_lumi
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooRealVar::e_pol_0
[#1] INFO:Obje

In [6]:
ws.Print("t")


RooWorkspace(ws) ws contents

variables
---------
(O_g1z_pos_1em05_0,O_ka_pos_1em05_0,O_la_pos_1em05_0,e_pol_0,g1z,ka,la,mu_signal,p_pol_0,total_lumi)

p.d.f.s
-------
RooMultiVarGaussian::multi_gauss_0[ x=(O_g1z_pos_1em05_0,O_ka_pos_1em05_0,O_la_pos_1em05_0) mu=(expectation_O_g1z_pos_1em05_0,expectation_O_ka_pos_1em05_0,expectation_O_la_pos_1em05_0) ] = 1
  RooFunctorBinding::expectation_O_g1z_pos_1em05_0[ function=0x228753d0 vars=(lumi_0,e_pol_0,p_pol_0,g1z,ka,la,mu_signal) ] = -131325
    RooProduct::lumi_0[ 1 * total_lumi ] = 2000
  RooFunctorBinding::expectation_O_ka_pos_1em05_0[ function=0x22fda350 vars=(lumi_0,e_pol_0,p_pol_0,g1z,ka,la,mu_signal) ] = -326754
    RooProduct::lumi_0[ 1 * total_lumi ] = 2000
  RooFunctorBinding::expectation_O_la_pos_1em05_0[ function=0x2305f350 vars=(lumi_0,e_pol_0,p_pol_0,g1z,ka,la,mu_signal) ] = -161778
    RooProduct::lumi_0[ 1 * total_lumi ] = 2000



In [7]:
model = ws.pdf("multi_gauss_0")
# model = ws.pdf("sim_pdf")

In [8]:
print([o.GetName() for o in obs_pars])

['O_g1z_pos_1em05_0', 'O_ka_pos_1em05_0', 'O_la_pos_1em05_0']


In [9]:
ds = ROOT.RooStats.AsymptoticCalculator.GenerateAsimovData(model, obs_pars)
ds.Print("v")

DataStore CountingAsimovData0 (CountingAsimovData0)
  Contains 1 entries
  Observables: 
    1)  O_g1z_pos_1em05_0 = -131325  L(-135092 - -127558)  "O_g1z_pos_1em05_0"
    2)   O_ka_pos_1em05_0 = -326754  L(-330045 - -323462)  "O_ka_pos_1em05_0"
    3)   O_la_pos_1em05_0 = -161778  L(-166859 - -156697)  "O_la_pos_1em05_0"


In [10]:
toy_data = fit_handler.make_toy_obs_mult(ilc_250_h20_lumi, ilc_250_h20_hel)
print(toy_data)
toy_data = fit_handler.make_toy_obs_mult(ilc_250_h20_lumi, ilc_250_h20_hel, parameter_set={"g1z": 0.001, "ka": 0., "la": 0.})
print(toy_data)

[[-209628.96454317894, -437774.29268074036, -229088.14151385502], [-4512.886556540327, -15552.797415210198, -8996.942641946152], [-8127.80318375827, -20688.51306658026, -10021.498199824433], [-1515.3354927551231, -7854.470969646526, -3333.8460908676893]]
[[-211331.0356960001, -437856.76851864054, -227910.98116304353], [-4564.249542567695, -15519.127984066761, -8972.252170233553], [-8188.953400174445, -20683.25028067477, -9983.391598275373], [-1530.0219608972411, -7848.499135602722, -3326.5152868806363]]


In [11]:
fit_res = model.fitTo(ds, Save=True)

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 246.507 μs
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       24.19490062 Edm =    7.14602917e-27 NCalls =     13
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 24.19490062
  Edm           : 7.14602917e-27
  Internal parameters:	[                0                0                0]	
  Internal gradient  :	[ -3.528754831e-11  2.646566123e-11  1.764377416e-11]	
  Internal covariance matrix:
[[  6.9805505e-06              0              0]
 [              0  2.5209285e-05              0]
 [              0              0  7.1778605e-06]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.001 with call limit = 1500
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =       24.19490062 Edm =    7.14602917e-27 NCalls =     13
Warning in <Minuit2>: VariableMetricBuilder No improvement

[#1] INFO:Fitting -- RooAbsPdf::fitTo(multi_gauss_0_over_multi_gauss_0_Int[O_g1z_pos_1em05_0,O_ka_pos_1em05_0,O_la_pos_1em05_0]) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using generic CPU library compiled with no vectorizations
[#1] INFO:Fitting -- Creation of NLL object took 7.9251 ms
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_multi_gauss_0_over_multi_gauss_0_Int[O_g1z_pos_1em05_0,O_ka_pos_1em05_0,O_la_pos_1em05_0]_CountingAsimovData0) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
Minuit2Minimizer: Minimize with max-calls 1500 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = 24.1949006238567748
Edm   = 1.02332748709335346e-26
Nfcn  = 40
g1z	  = 0	 +/-  0.00139167	(limited)
ka	  = 0	 +/-  0.00189594	(limited)
la	  = 0	 +/-  0.00136195	(limited)
[#1] INFO:Minimization -- RooAbs

In [12]:
fit_res.covarianceMatrix().Print()


3x3 matrix is as follows

     |      0    |      1    |      2    |
--------------------------------------------
   0 |  1.937e-06  -6.922e-07   1.322e-06 
   1 | -6.922e-07   3.595e-06  -4.003e-08 
   2 |  1.322e-06  -4.003e-08   1.855e-06 



In [13]:
nll = model.createNLL(ds, EvalBackend="cpu")

[#1] INFO:Fitting -- RooAbsPdf::fitTo(multi_gauss_0_over_multi_gauss_0_Int[O_g1z_pos_1em05_0,O_ka_pos_1em05_0,O_la_pos_1em05_0]) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- Creation of NLL object took 217.246 μs


In [14]:
nll_minimizer = ROOT.RooMinimizer(nll)

[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_multi_gauss_0_over_multi_gauss_0_Int[O_g1z_pos_1em05_0,O_ka_pos_1em05_0,O_la_pos_1em05_0]_CountingAsimovData0) Summation contains a RooNLLVar, using its error level


In [15]:
%time
nll_minimizer.migrad()

CPU times: user 5 μs, sys: 1e+03 ns, total: 6 μs
Wall time: 11.2 μs


0

Minuit2Minimizer: Minimize with max-calls 1500 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = 24.1949006238567748
Edm   = 0
Nfcn  = 29
g1z	  = 0	 +/-  0.00139167	(limited)
ka	  = 0	 +/-  0.00189594	(limited)
la	  = 0	 +/-  0.00136195	(limited)


Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 250.94 μs
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       24.19490062 Edm =                 0 NCalls =     13
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 24.19490062
  Edm           : 0
  Internal parameters:	[                0                0                0]	
  Internal gradient  :	[                0                0                0]	
  Internal covariance matrix:
[[  6.9805504e-06              0              0]
 [              0  2.5209284e-05              0]
 [              0              0  7.1778604e-06]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.001 with call limit = 1500
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =       24.19490062 Edm =                 0 NCalls =     13
Info in <Minuit2>: MnHesse Done after 301.846 μs
Info in <Minuit2>: Vari

In [16]:
pll0 = nll.createProfile({coupling_pars[0]})
pll1 = nll.createProfile({coupling_pars[1]})
pll2 = nll.createProfile({coupling_pars[2]})

In [17]:
frame0 = coupling_pars[0].frame(Range=(-0.004, 0.004), Title=f";{list(conf["parameters"])[0]};-#Delta NLL")
frame1 = coupling_pars[1].frame(Range=(-0.004, 0.004), Title=f";{list(conf["parameters"])[1]};-#Delta NLL")
frame2 = coupling_pars[2].frame(Range=(-0.004, 0.004), Title=f";{list(conf["parameters"])[2]};-#Delta NLL")
nll.plotOn(frame0, ShiftToZero=True)
nll.plotOn(frame1, ShiftToZero=True)
nll.plotOn(frame2, ShiftToZero=True)

In [18]:
pll0.plotOn(frame0, LineColor="r")
pll1.plotOn(frame1, LineColor="r")
pll2.plotOn(frame2, LineColor="r")

[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[g1z]) Creating instance of MINUIT
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_multi_gauss_0_over_multi_gauss_0_Int[O_g1z_pos_1em05_0,O_ka_pos_1em05_0,O_la_pos_1em05_0]_CountingAsimovData0) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[g1z]) determining minimum likelihood for current configurations w.r.t all observable
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[g1z]) minimum found at (g1z=0)
..........................................................................................................................................................................................................
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[ka]) Creating instance of MINUIT
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_multi_gauss_0_over_multi_gaus

In [19]:
c0 = ROOT.TCanvas()
frame0.SetMinimum(0)
frame0.SetMaximum(2)
frame0.Draw()
c0.Draw()
# c0.SaveAs("plots/fit/ll_pll.pdf(")
c0.SaveAs("plots/fit/ll_pll_g1z.pdf")

c1 = ROOT.TCanvas()
frame1.SetMinimum(0)
frame1.SetMaximum(2)
frame1.Draw()
c1.Draw()
# c1.SaveAs("plots/fit/ll_pll.pdf")
c1.SaveAs("plots/fit/ll_pll_ka.pdf")

c2 = ROOT.TCanvas()
frame2.SetMinimum(0)
frame2.SetMaximum(2)
frame2.Draw()
c2.Draw()
# c2.SaveAs("plots/fit/ll_pll.pdf)")
c2.SaveAs("plots/fit/ll_pll_la.pdf")

Info in <TCanvas::Print>: pdf file plots/fit/ll_pll_g1z.pdf has been created
Info in <TCanvas::Print>: pdf file plots/fit/ll_pll_ka.pdf has been created
Info in <TCanvas::Print>: pdf file plots/fit/ll_pll_la.pdf has been created
